In [1]:
import torch
from transformers import AutoModelForCausalLM
from utils.data_loader import create_cached_tsmixup_datasets
from load_cached_features import *
import pandas as pd
import numpy as np
import einops
import time 
device = 'cuda:3'

In [2]:
# Loading tsmixup dataset
train_dataset, val_dataset = create_cached_tsmixup_datasets(
        max_samples=300000,
        context_length=512,
        prediction_length=128, # 1 or 128
        num_workers=16,
        cache_dir="/extra/datalab_scratch0/ctadler/time_series_models/mechanistic_interpretability/data/tsmixup_cache/",
        processed_cache_path="/extra/datalab_scratch0/ctadler/time_series_models/mechanistic_interpretability/data/tsmixup_cache/tsmixup_processed_300000_512_128.pkl",
        batch_size=4000
    )

def load_dataset(dataset, ts=1000, pred_length=1, ctx_len=512):
    if dataset == 'tsmixup':
        x = []
        y = []
        for i in range(ts) if isinstance(ts, int) else ts:
            val_dict = val_dataset[i]
            x.append(val_dict['past_values'])
            y.append(val_dict['future_values'])
        x = torch.stack(x)[:, -ctx_len:]
        y = torch.stack(y)[:,:pred_length]
        
    else:
        dataset_path = f"/extra/datalab_scratch0/ctadler/time_series_models/ts_foundation_calibration/data/{dataset}/y_{dataset}.csv"
        timestamp_column = "ds"

        data = pd.read_csv(
            dataset_path,
            parse_dates=[timestamp_column],
            index_col=0
        )

        x = []
        for id, vals in data.groupby('unique_id'):
            x.append(torch.from_numpy(vals['y'].to_numpy(np.float32)))
        x = torch.stack(x)

    if dataset != 'tsmixup':
        y = x[:,ctx_len:ctx_len+pred_length]
        x = x[:,:ctx_len]
    return x, y


In [14]:
# load pretrain model
model = AutoModelForCausalLM.from_pretrained('qcw2333/YingLong_110m', trust_remote_code=True,torch_dtype=torch.bfloat16).to(device).eval()

# prepare input
# batch_size, lookback_length = 1, 2880
# seqs = torch.randn(batch_size, lookback_length).bfloat16().cuda()
batch_size = 512
context_len = 512
x, y = load_dataset('tsmixup', np.random.randint(0, 10000, batch_size), pred_length=128, ctx_len=context_len)
x = x.to(device).to(torch.bfloat16)
# print(f"patch_len: {model.config.patch_size}")
pred_len = 128
forecast_length = 2048
patch_len = 32
d_model = 768
n_quantiles = 99
decoder_out = torch.zeros((batch_size, (pred_len)//patch_len, d_model))
loc_scale = torch.zeros((batch_size, 2))
token_masks = []
def save_decoder_hook(module, input, output):
    # print(f"transformer out: {input[0].shape}")
    # print(f"head_out shape: {output.shape}")
    # head_out = einops.rearrange(output,'b c (l1 l2) -> b c l1 l2', l2 = 99)
    # print(f"head out: {head_out.shape} masked head: {head_out[:,token_masks,:,:].shape}")
    # model_out = einops.rearrange(head_out[:,token_masks,:,:], 'b l c d -> b (l c) d')
    # print(f"model_out: {model_out.shape} model out: {model_out[:,:pred_len].shape}")
    # print(f"pred mask: {context_len//patch_len},{(context_len+pred_len)//patch_len}")
    decoder_out[:] = input[0][:,context_len//patch_len:(context_len+pred_len)//patch_len].detach().cpu()

def save_tokenizer_hook(module, input, output):
    x, x_raw, masks, mean, std, _ = output
    # print(f"mean {mean.shape}, std {std.shape}")
    # print(f"masks {len(masks), masks}")
    loc_scale[:,0] = mean.flatten().detach().cpu()
    loc_scale[:,1] = std.flatten().detach().cpu()
    # token_masks.extend(masks)

# generate forecast
model.lm_head.register_forward_hook(save_decoder_hook)
model.tokenizer.register_forward_hook(save_tokenizer_hook)

start_time = time.time()
with torch.no_grad():
    output = model.generate(x, future_token=forecast_length)
print(f"Time taken: {(time.time()-start_time):4f}")

# print(f" output_shape {output.shape}")

Time taken: 0.166243
